###### lien vers la doc utilisé : https://www.datacamp.com/fr/tutorial/tutorial-postgresql-python
https://docs.postgresql.fr/search.php

In [1]:
#pip install psycopg2-binary

In [2]:
import psycopg2
import pandas as pd
import numpy as np

### ##

In [3]:
#connexion avec la postgres
#conn = psycopg2.connect(
#    dbname="gtfs_db",
#    user="postgres",
#    password="testdata"
#)
#conn.close()

In [4]:
try:
    conn = psycopg2.connect(
        database="gfts_db",  
        user="postgres",
        password="testdata"
    )
    print("Connection successful!")
    #conn.close()
except psycopg2.Error as e:
    print(f"Error connecting: {e}")

Connection successful!


##### il faut créer un curseur pour permettre les exécutions de postgres avec python

In [5]:
# le curseur pour exécuter les requêtes
cur = conn.cursor()

In [6]:
create_table_query = """
CREATE TABLE IF NOT EXISTS horaires_gtfs_complet (
    id SERIAL PRIMARY KEY,
    trip_id TEXT,
    arrival_time TIME,
    departure_time TIME,
    stop_id TEXT,
    stop_sequence INTEGER,
    pickup_type INTEGER,
    drop_off_type INTEGER,
    shape_dist_traveled FLOAT,
    stop_headsign TEXT,
    route_id TEXT,
    service_id TEXT,
    trip_headsign TEXT,
    direction_id INTEGER,
    block_id TEXT,
    shape_id TEXT,
    wheelchair_accessible INTEGER,
    bikes_allowed INTEGER,
    agency_id TEXT,
    route_short_name TEXT,
    route_long_name TEXT,
    route_desc TEXT,
    route_type INTEGER,
    route_color TEXT,
    route_text_color TEXT,
    stop_code TEXT,
    stop_name TEXT,
    stop_desc TEXT,
    stop_lat FLOAT,
    stop_lon FLOAT,
    location_type INTEGER,
    parent_station TEXT,
    wheelchair_boarding INTEGER
);
"""

cur.execute(create_table_query)
conn.commit()
print("Table GTFS complète créée avec succès.")

Table GTFS complète créée avec succès.


In [7]:
gtfs_full_schedule=pd.read_csv('gtfs_full_schedule')


C:\Users\Gbemissola\AppData\Local\Temp\ipykernel_14472\2216559479.py:1: DtypeWarning: Columns (19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  gtfs_full_schedule=pd.read_csv('gtfs_full_schedule')


In [8]:
          # Assure-toi que les heures sont bien des objets time
df = gtfs_full_schedule.copy()
df['departure_time'] = pd.to_datetime(df['departure_time'], errors='coerce').dt.time
df['arrival_time'] = pd.to_datetime(df['arrival_time'], errors='coerce').dt.time

C:\Users\Gbemissola\AppData\Local\Temp\ipykernel_14472\13809074.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['departure_time'] = pd.to_datetime(df['departure_time'], errors='coerce').dt.time
C:\Users\Gbemissola\AppData\Local\Temp\ipykernel_14472\13809074.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['arrival_time'] = pd.to_datetime(df['arrival_time'], errors='coerce').dt.time


In [9]:
#df['departure_time']

In [10]:
#il faut remplacer les NaN par none parceque Postgres ne support pas ça
df_cleaned = df.replace({pd.NA: None, np.nan: None})

#### Pour accélérer l'insertion : au lieu  de faire ligne par ligne on réalise l'insertion par bach ce qui reduit le temps de travail du script , l'utilisation de la mémoire et le nombbre de fois la requête est exécuté

In [11]:
from psycopg2.extras import execute_batch

In [12]:
# Préparer l'insert
insert_query = """
    INSERT INTO horaires_gtfs_complet (
        trip_id, arrival_time, departure_time, stop_id, stop_sequence,
        pickup_type, drop_off_type, shape_dist_traveled, stop_headsign,
        route_id, service_id, trip_headsign, direction_id, block_id,
        shape_id, wheelchair_accessible, bikes_allowed,
        agency_id, route_short_name, route_long_name, route_desc,
        route_type, route_color, route_text_color, stop_code,
        stop_name, stop_desc, stop_lat, stop_lon, location_type,
        parent_station, wheelchair_boarding
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data_to_insert = list(df_cleaned[[
    'trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
    'pickup_type', 'drop_off_type', 'shape_dist_traveled', 'stop_headsign',
    'route_id', 'service_id', 'trip_headsign', 'direction_id', 'block_id',
    'shape_id', 'wheelchair_accessible', 'bikes_allowed',
    'agency_id', 'route_short_name', 'route_long_name', 'route_desc',
    'route_type', 'route_color', 'route_text_color', 'stop_code',
    'stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'location_type',
    'parent_station', 'wheelchair_boarding'
]].itertuples(index=False, name=None))

# Insertion rapide
execute_batch(cur, insert_query, data_to_insert)

conn.commit()


#### dès a présent on va chercher à faire la connexion à l'API pour avoir les données en temps réel.Dans le cas du fichier gfts les DTR  sous format .pbf Protocol Buffers

#### le décodage est l'étape cruciale pour transformer un flux de données brutes et potentiellement binaires en une représentation structurée et exploitable des informations en temps réel sur les transports publics. Sans cette étape, les données GTFS-RT seraient inutiles pour la plupart des applications qui cherchent à afficher des informations en temps réel aux utilisateurs, à analyser la performance du réseau, ou à prendre des décisions opérationnelles.

In [1]:
import requests
from google.transit import gtfs_realtime_pb2

# URL du flux GTFS-RT Grand Poitiers
url = "https://data.grandpoitiers.fr/data-fair/api/v1/datasets/2gwvlq16siyb7d9m3rqt1pb1/metadata-attachments/poitiers.pbf"

# Récupérer les données
response = requests.get(url)

if response.status_code == 200:
    # Décoder le fichier GTFS-RT
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)

    # Afficher le contenu complet sous forme structurée
for entity in feed.entity:
    print(entity)  # Affiche chaque entité sous forme structurée
else:
    print(f"Erreur lors de la récupération des données : {response.status_code}")

id: "RT|517416|Vitalis:VehicleJourney::C517416:LOC"
trip_update {
  trip {
    trip_id: "517416"
    start_date: "20250528"
    schedule_relationship: SCHEDULED
    route_id: "156"
    direction_id: 0
  }
  stop_time_update {
    stop_sequence: 1
    departure {
      delay: 0
      time: 1748411460
    }
    stop_id: "20012"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 2
    arrival {
      delay: 0
      time: 1748411543
    }
    departure {
      delay: 9
      time: 1748411552
    }
    stop_id: "20885"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 3
    arrival {
      delay: 0
      time: 1748411604
    }
    departure {
      delay: 0
      time: 1748411604
    }
    stop_id: "20883"
    schedule_relationship: SCHEDULED
  }
  stop_time_update {
    stop_sequence: 4
    arrival {
      delay: -29
      time: 1748411631
    }
    departure {
      delay: -18
      time: 1748411642
    }
    stop_id: "20166"
  

In [15]:

import requests
from google.transit import gtfs_realtime_pb2
from datetime import datetime  # ✅ import nécessaire


response = requests.get(url)

if response.status_code == 200:
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)

    for entity in feed.entity:
        if entity.HasField("trip_update"):
            trip = entity.trip_update.trip
            print(f"🚌 Bus {trip.trip_id} - Ligne {trip.route_id}")
            for stop in entity.trip_update.stop_time_update:
                stop_id = stop.stop_id
                stop_sequence = stop.stop_sequence
                delay = stop.arrival.delay if stop.HasField("arrival") else "Inconnu"
                timestamp = stop.arrival.time if stop.HasField("arrival") else None

                # ✅ Corrigé
                if timestamp:
                    arrival_time = datetime.utcfromtimestamp(timestamp).strftime('%H:%M:%S')
                else:
                    arrival_time = "Inconnu"

                print(f"   🚏 Arrêt {stop_id} ({stop_sequence}) - Heure prévue: {arrival_time} - Retard: {delay} sec")


🚌 Bus 528519 - Ligne 41
   🚏 Arrêt 20082 (1) - Heure prévue: Inconnu - Retard: Inconnu sec
   🚏 Arrêt 20882 (2) - Heure prévue: 23:47:09 - Retard: 0 sec
   🚏 Arrêt 20523 (3) - Heure prévue: 23:47:56 - Retard: 0 sec
   🚏 Arrêt 20521 (4) - Heure prévue: 23:48:50 - Retard: 0 sec
   🚏 Arrêt 20955 (5) - Heure prévue: 23:50:00 - Retard: 0 sec
   🚏 Arrêt 20705 (6) - Heure prévue: 23:50:59 - Retard: 0 sec
   🚏 Arrêt 21750 (7) - Heure prévue: 23:51:51 - Retard: 0 sec
   🚏 Arrêt 20074 (8) - Heure prévue: 23:53:00 - Retard: 0 sec
   🚏 Arrêt 20072 (9) - Heure prévue: 23:54:00 - Retard: 0 sec
   🚏 Arrêt 20070 (10) - Heure prévue: 23:54:57 - Retard: 0 sec
   🚏 Arrêt 20068 (11) - Heure prévue: 23:55:59 - Retard: 0 sec
   🚏 Arrêt 20044 (12) - Heure prévue: Inconnu - Retard: Inconnu sec
   🚏 Arrêt 20020 (13) - Heure prévue: 23:59:24 - Retard: 24 sec
   🚏 Arrêt 20040 (14) - Heure prévue: 00:00:17 - Retard: 24 sec
   🚏 Arrêt 20002 (15) - Heure prévue: 00:02:13 - Retard: 24 sec
   🚏 Arrêt 20137 (16) - Heu

C:\Users\Gbemissola\AppData\Local\Temp\ipykernel_14472\473854213.py:24: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  arrival_time = datetime.utcfromtimestamp(timestamp).strftime('%H:%M:%S')


#### API pour la météo

In [16]:
import requests
import pandas as pd
from datetime import datetime

def get_weather(latitude, longitude, date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": date,
        "end_date": date,
        "hourly": "temperature_2m,precipitation,weathercode",
        "timezone": "Europe/Paris"
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Conversion en DataFrame
    df = pd.DataFrame(data['hourly'])
    df['time'] = pd.to_datetime(df['time'])
    return df

# Exemple : Poitiers le 2024-05-10
df_meteo = get_weather(46.58, 0.34, "2024-05-10")
print(df_meteo.head())


                 time  temperature_2m  precipitation  weathercode
0 2024-05-10 00:00:00            13.5            0.0            0
1 2024-05-10 01:00:00            13.1            0.0            0
2 2024-05-10 02:00:00            12.6            0.0            0
3 2024-05-10 03:00:00            12.2            0.0            0
4 2024-05-10 04:00:00            11.8            0.0            0
